In [151]:
%load_ext autoreload
%autoreload 2

from langchain.tools import BaseTool
import requests
from pydantic import BaseModel, Field
from typing import List, Optional
import os
import json

#from backend.custom_tools import adeptid as aid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:

ADEPT_ID_API_KEY = os.getenv("ADEPTID_API_KEY")

degree_type = [
    "DOCTORATE",
    "MASTERS",
    "BACHELORS",
    "ASSOCIATES",
    "HIGH_SCHOOL",
    "CERTIFICATE"
]

# Define the input schema for the tool
class WorkHistory(BaseModel):
    title: str
    start_date: Optional[str]
    end_date: Optional[str]

class CandidateEducation(BaseModel):
    level: str #this is an enum, must come from list 
    subject: Optional[str]
    start_date: Optional[str] #think about adding data validation as MM/YYYY
    end_date: Optional[str] #think about adding data validation as MM/YYYY, must be > start_date
    institution: Optional[str]
    gpa: Optional[float]
    summary: Optional[str]

class Candidate(BaseModel):
    id: str
    work_history: List[WorkHistory]
    #education: Optional[List[CandidateEducation]]
    skills: Optional[List[str]]

#TODO define job object
class Job(BaseModel):
    title: str
    id: str #this has to be unique
    naics: Optional[str]
    employer_name: Optional[str]
    skills: Optional[List[str]]
    #education: str #TODO: define education object
    

class AdeptIDSkillSearchInput(BaseModel):
    skills: List[str]
    result_count: int = 1
    offset: int = 1

class AdeptIDDestionationOccupationRecommendationInput(BaseModel):
    candidates: List[Candidate]
    limit: int = 10
    offset: int = 1

class AdeptIDJobRecommendationInput(BaseModel):
    #skill_count: int = 0
    destination_jobs: List[str]
    limit: int = 1000
    offset: int = 0


class AdeptIDEvaluateJobsInput(BaseModel):
    skill_count: int = 1
    destination_jobs: List[Job]
    candidate: Candidate

In [153]:
class AdeptIDSkillSearch(BaseTool):
    name = "AdeptIDSkillSearch"
    description = "Searches for skills and returns the top results."

    input = AdeptIDSkillSearchInput

    def _run(self, **kwargs):
        url = "https://api.adept-id.com/v2/skill"
        api_key = ADEPT_ID_API_KEY
        headers = {
            "X-API-KEY": f"{api_key}"
        }
        payload = input.json()
        #TODO: check whether this is GET or POST
        response = requests.post(url, headers=headers, data=payload)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": response.json()}

class AdeptIDDestinationOccupationRecommendation(BaseTool):
    name = "AdeptIDDestinationOccupationRecommendation"
    description = "Recommends occupations and career paths to a candidate based on their skills and interests."

    input = AdeptIDDestionationOccupationRecommendationInput

    def _run(self, **kwargs):
        url = "https://api.adept-id.com/v2/recommend-destination-occupation"
        api_key = ADEPT_ID_API_KEY
        headers = {
            "X-API-KEY": f"{api_key}"
        }
        payload = json.loads(json.dumps(kwargs))

        response = requests.post(url, headers=headers, data=payload)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": response.json()}

# Define the tool class
class AdeptIDJobRecommendation(BaseTool):
    name = "AdeptIDJobRecommendation"
    description = "Recommends specific jobs for a candidate based on their destination occupation and skills."

    input = AdeptIDEvaluateJobsInput

    def _run(self, **kwargs):
        url = "https://api.adept-id.com/v2/evaluate-jobs"
        api_key = ADEPT_ID_API_KEY
        headers = {
            "X-API-KEY": f"{api_key}"
        }
        #payload = input.json()
        #payload = kwargs
        payload = json.loads(json.dumps(kwargs))
        print(payload)

        response = requests.post(url, headers=headers, json=payload)
        print(response.json())
        
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": response.json()}


In [154]:
import copy
def remove_none_types(candidate_object):
    candidate_object_no_none = copy.deepcopy(candidate_object)
    
    def remove_none_types_recursive(candidate_object_no_none):
        keys_to_remove = []
        for key, value in candidate_object_no_none.items():
            if value is None:
                keys_to_remove.append(key)
            elif isinstance(value, dict):
                remove_none_types_recursive(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        remove_none_types_recursive(item)

        for key in keys_to_remove:
            del candidate_object_no_none[key]

        return candidate_object_no_none

    candidate_object_no_none = remove_none_types_recursive(candidate_object_no_none)
            
    return candidate_object_no_none

In [155]:
#test inputs

skills_data =  ["carpentry", "communication", "cooking", "python"]

job_data = AdeptIDJobRecommendationInput(
    destination_jobs = ["Carpenter", "Computer programmer"],
)

occupation_data = AdeptIDDestionationOccupationRecommendationInput(
        candidates=[
            Candidate(
                id="Piya Gupta",
                work_history=[
                    WorkHistory(title="Retail Sales Associate", start_date="12/2019", end_date="4/2022"),
                    WorkHistory(title="Graphic Designer", start_date="5/2022", end_date="4/2024")
                ],
                education=[
                    Education(
                        level="ASSOCIATES", degree_type="ASSOCIATE OF ARTS", subject="Business",
                        start_date="09/2020", end_date="06/2022",
                        institution="Northern Virgina Community College", gpa=3.5,
                        summary="Completed Associate's in Business with a concentration in marketing and high honors graduation"
                    )
                ],
                skills=["Graphic Design", "Adobe Photoshop", "Illustration", "Marketing", "Logo Design", "Merchandising", "Writing"]
            )
        ],
        limit=10,
        offset=1,
        skill_count=5
    )

skills_data = AdeptIDSkillSearchInput(
    skills=skills_data
)

In [156]:
skill_search = AdeptIDSkillSearch()
occupation_recommendation = AdeptIDDestinationOccupationRecommendation() 
job_recommendation = AdeptIDJobRecommendation()

In [157]:
 
# Example input
input_data1 = AdeptIDJobRecommendationInput(
    destination_jobs=["Carpenter", "Computer programmer"],  # Add this line
    candidates=[
        Candidate(
            id="Piya Gupta",
            work_history=[
                WorkHistory(title="Retail Sales Associate", start_date="12/2019", end_date="4/2022"),
                WorkHistory(title="Graphic Designer", start_date="5/2022", end_date="4/2024")
            ],
            education=[
                Education(
                    level="ASSOCIATES", degree_type="ASSOCIATE OF ARTS", subject="Business",
                    start_date="09/2020", end_date="06/2022",
                    institution="Northern Virgina Community College", gpa=3.5,
                    summary="Completed Associate's in Business with a concentration in marketing and high honors graduation"
                )
            ],
            skills=["Graphic Design", "Adobe Photoshop", "Illustration", "Marketing", "Logo Design", "Merchandising", "Writing"]
        )
    ],
    limit=10,
    offset=1,
    skill_count=5
)

input_data2 = AdeptIDJobRecommendationInput(
    destination_jobs=["Carpenter", "Computer programmer"],
    limit=10,
    offset=1
)

input_data3 = AdeptIDEvaluateJobsInput(
    destination_jobs=[Job(title="Carpenter", id="123", employer_name="ABC Corp", skills=["Carpentry", "CAD", "CAM"])],
    candidate=Candidate(id="Piya Gupta", work_history=[WorkHistory(title="Chief Data Nerd", start_date="12/2019", end_date="4/2022")], skills=["Graphic Design", "Adobe Photoshop", "Illustration", "Marketing", "Logo Design", "Merchandising", "Writing"])
)

input_data4 = AdeptIDEvaluateJobsInput(
    destination_jobs=[Job(title="Carpenter", id="123", employer_name="ABC Corp", skills=["Carpentry", "CAD", "CAM"])],
    candidate=Candidate(id="Piya Gupta",education=[CandidateEducation(level="ASSOCIATES", degree_type="ASSOCIATE OF ARTS", subject="Business", start_date="09/2020", end_date="06/2022", institution="Northern Virgina Community College", gpa=3.5, summary="Completed Associate's in Business with a concentration in marketing and high honors graduation")], work_history=[WorkHistory(title="Chief Data Nerd", start_date="12/2019", end_date="4/2022")], skills=["Graphic Design", "Adobe Photoshop", "Illustration", "Marketing", "Logo Design", "Merchandising", "Writing"])
)

In [158]:
# Test Job Recommendation
tool = AdeptIDJobRecommendation()
#input_data_dict = json.dumps(input_data3.dict())

input_data_dict = input_data3.dict()
input_data_dict = remove_none_types(input_data_dict)
result = tool.run(tool_input=input_data_dict)

{'skill_count': 1, 'destination_jobs': [{'title': 'Carpenter', 'id': '123', 'employer_name': 'ABC Corp', 'skills': ['Carpentry', 'CAD', 'CAM']}], 'candidate': {'id': 'Piya Gupta', 'work_history': [{'title': 'Chief Data Nerd', 'start_date': '12/2019', 'end_date': '4/2022'}], 'skills': ['Graphic Design', 'Adobe Photoshop', 'Illustration', 'Marketing', 'Logo Design', 'Merchandising', 'Writing']}}
{'classifications': {'Carpenter': {'code': '47-2031.00', 'common_name': 'Carpenters', 'common_name_singular': 'Carpenter', 'name': 'Carpenters'}, 'Chief Data Nerd': {'code': '15-2051.00', 'common_name': 'Data Scientists', 'common_name_singular': 'Data Scientist', 'name': 'Data Scientists'}}, 'jobs_scores': {'123': {'match_score': 0.23180915415287018, 'match_score_category': 'Low', 'skills': {'gaps': [{'category': 'Very Large', 'name': 'Carpentry', 'skill_category': 'Carpentry', 'skill_type': 'Specialized Skill', 'value': 1.0}], 'overlaps': [{'category': 'Very Large', 'name': 'Communications', 'sk

In [159]:
result

{'classifications': {'Carpenter': {'code': '47-2031.00',
   'common_name': 'Carpenters',
   'common_name_singular': 'Carpenter',
   'name': 'Carpenters'},
  'Chief Data Nerd': {'code': '15-2051.00',
   'common_name': 'Data Scientists',
   'common_name_singular': 'Data Scientist',
   'name': 'Data Scientists'}},
 'jobs_scores': {'123': {'match_score': 0.23180915415287018,
   'match_score_category': 'Low',
   'skills': {'gaps': [{'category': 'Very Large',
      'name': 'Carpentry',
      'skill_category': 'Carpentry',
      'skill_type': 'Specialized Skill',
      'value': 1.0}],
    'overlaps': [{'category': 'Very Large',
      'name': 'Communications',
      'skill_category': 'Communication',
      'skill_type': 'Baseline Skill',
      'value': 1.0}]}}},
 'unknown_skills': ['cad', 'cam'],
 'operation_id': '405230f49d2e4638b531a545103c56d9'}